# **Numerical Solution of the Schrödinger Equation for a 1D Quantum Well**


<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/quantum-mechanics/1quantumwell.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

<p style="text-align: justify;font-size:15px">
    This notebook solves numerically the quantum-mechanical problem
    of a single rectangular one-dimensional quantum well,
    and displays interactively the eigenfunctions 
    (plotted at the height of the corresponding eigenvalues).
</p>

<details close>
    <summary style="font-size: 20px"><b>Sub-goals</b></summary>
    <ol style="text-align: justify;font-size:15px">
        <li> Understand the Schrödinger equation and all its terms.</li>        
        <li> Learn how to solve the Schrödinger equation by matrix diagonalization.</li>
        <li> Examine the effect of quantum tunnelling of a wavefunction in regions
            where the energy is lower than the potential.</li>
    </ol>

</details>

## **Background theory**

<p style="text-align: justify;font-size:15px">
    The Schrödinger equation is the core of quantum mechanics. 
    Solving it, we can obtain chemical and physical properties of
    molecular and crystal systems. However, solving the Schrödinger equation
    for realistic systems is computationally very expensive. The analytical 
    solution is only available for simple systems, such as a hydrogen atom.
    <br>
    In this notebook, we demonstrate how to solve the (time-independent) Schrödinger equation 
    numerically for a one-dimensional (1D) quantum well. The eigenvalues and 
    eigenfunctions (wavefunctions) are solved by diagonalization of the
    Hamiltonian, written in matrix form on a regular grid along the 1D axis.
    Eigenfunctions and eigenvalues are shown interactively in the figure below.
</p>

<details close>
<summary style="font-size: 20px">Schrödinger equation</summary>
<p style="text-align: justify;font-size:15px">
    The Schrödinger equation is named after the Austrian-Irish physicist Erwin Schrödinger.
    The (time-independent) Schrödinger equation for a one-dimensional system is:
</p>

$\large -\frac{\hbar^2}{2m}\frac{d^2}{d x^2} \psi_n(x) + V(x)\psi_n(x) = E_n\psi_n(x)$ &nbsp;&nbsp;&nbsp;&nbsp;(1)
  
<p style="text-align: justify;font-size:15px">
    where $\psi_n(x)$ is the wavefunction of the 1D system, $\hbar$ is the 
    reduced Planck constant, $V(x)$ is the potential energy, $m$ is the mass 
    of the particle, and $E_n$ is the energy of the system.
</p>
    
<p style="text-align: justify;font-size:15px">
    By solving this eigenvalue problem, we can obtain 
    the wavefunctions (or eigenfunctions) $\psi_n(x)$ and the corresponding
    eigenenergies $E_n$, labelled by an integer index $n$.
</p>
    
</details>

<details close>
<summary style="font-size: 20px">Numerical method</summary>
    
<p style="text-align: justify;font-size:15px">
    If we define the Hamiltonian operator $\hat H$ as
</p>
    
$\large \hat H = \frac{\hbar^2}{2m}\frac{d^2}{d x^2} + \hat V$,
    
<p style="text-align: justify;font-size:15px">
    the Schrödinger equation can be written as:
</p>
    
$\large \hat H \psi_n = E_n \psi_n$
    
<p style="text-align: justify;font-size:15px">
    In this form, it is clear that the Schrödinger equation is
    a typical eigenvalue equation. 
    If we discretize the $x$ axis on a regular grid of $N$ points
    $(x_0,x_1,x_2,\ldots,x_{N-1})$, then the wavefunction
    $\psi_n(x)$ can be written as a vector:
    $\psi_n(x) = [\psi_n(x_0),\psi_n(x_1),\ldots,\psi_n(x_{N-1})]$.
    In turn, we can discretize the Hamiltonian operator $\hat H$ as a matrix,
    and then solve the equation by numerical matrix diagonalization.
</p>
    
<p style="text-align: justify;font-size:15px">    
    Let us briefly discuss how to diagonalize $\hat H$, that is the sum of two terms.
    Discretizing the potential term is easy, since the potential energy is local.
    Therefore, the operation $\hat V\psi_n$ can be written as simply multiplying,
    at each grid point $x_i$, the wavefunction by the value of the potential at $x_i$
    (i.e., $V(x_i)$):
</p>    
    
$\hat V\psi = [V(x_0)\psi_n(x_0),V(x_1)\psi_n(x_1),\ldots,V(x_{N-1})\psi_n(x_{N-1})]$,
    
<p style="text-align: justify;font-size:15px">
    that is, the $V$ operator is a diagonal matrix, where the diagonal values are obtained
    by discretizing the potential energy $V$ on the same grid.
</p>
    
<p style="text-align: justify;font-size:15px">    
    In order to discretize the kinetic-energy term, we need to first understand how
    to discretize a second-derivative operator.
    If we consider a generic (discretized) 1D function $f(x) = [f_0,f_1,\ldots,f_{N-1}]$, 
    we want to write an approximation for its (discretized)
    second derivative $f''(x) = [f''_0,f''_1,\ldots,f''_{N-1}]$.
    We can do this by approximating the derivative:
</p>
    
$\large f''(x) = \lim_{\delta \rightarrow 0} \frac{f'(x+\delta/2)- f'(x-\delta/2))}{\delta} = \lim_{\delta \rightarrow 0} \frac{f(x+\delta) - 2f(x) + f(x-\delta)}{\delta^2} \approx \frac{f(x+\Delta) - 2f(x) + f(x-\Delta)}{\Delta^2}$,
 
<p style="text-align: justify;font-size:15px">
    where $\Delta$ is the distance between to neighboring grid points ($\Delta=x_1 - x_0$).
    Using this result, we can now write the second-derivative operator in matrix form as:
</p>

$\large
\begin{pmatrix}f''_0 \\ f''_1 \\ f''_2 \\\vdots \\ f''_{N-1}\end{pmatrix} = \frac{1}{\Delta^2}
\begin{pmatrix} -2 & 1 & 0 & 0 & \\ 1 & -2 & 1 & 0 & \\ 
0& 1 & -2 & 1 &  \\ &  & \ddots & \ddots & \ddots &\\
&  & & 1 & -2 \end{pmatrix}\begin{pmatrix}f_0 \\ f_1 \\ f_2 \\\vdots \\ f_{N-1}\end{pmatrix}
$

<p style="text-align: justify;font-size:15px">
    Putting all results together, the Hamiltonian operator $\hat H$ can be thus
    written as:
</p>
    
$\large
\hat H = 
\begin{pmatrix} -2C+ V_0 & C & 0 & & \\ C & -2C + V_1 & C & & \\ 0 & C & -2C + V_2 & & \\ & & &\ddots & \\ &&&&-2C + V_{N-1}\end{pmatrix}
$

<p style="text-align: justify;font-size:15px">
    where $C = -\frac{\hbar^2}{2 m \Delta^2}$.
</p>
    
<p style="text-align: justify;font-size:15px">    
    Using this expression, is now easy to write a numerical form for $\hat H$ for any potential
    term $V$ and use numerical routines to diagonalize the matrix and obtain eigenvalues and
    eigenfunctions. In this example, we use a square-well potential.
</p>

</details>

## **Tasks and exercises**

<ol style="text-align: justify;font-size:15px">
    <li> Investigate the wavefunctions and eigenvalues with different depth and width of 
        the well potential.
    <details style="color: blue">
    <summary>Hints</summary>
        One can tune the width and depth of the well potential with the slider. 
        The deeper and wider the well potential is, the more wavefunctions you will obtain.
        Note that the "zero" of the wavefunction is shifted, for every wavefunction, at the
        energy of the corresponding eigenvalue.
    </details>
    </li>
    <li> Use this notebook to explain the concept of quantum confinement.
    <details style="color: blue">
    <summary>Hints</summary>
        In a classical potential, the lowest energy state would be at the bottom of the well.
        In quantum mechanics, the lowest energy state has a higher energy than the bottom of the
        quantum well, as you can see in the figure below. The reason for this energy difference
        is what is typically called quantum confinement effect. Observe also how the 
    </details>
    </li>    
    <li> Why is the diagonalization of the Hamiltonian matrix key to solve the Schrödinger equation?
    <details style="color: blue">
    <summary>Hints</summary>
        As shown in the background theory section, the Schrödinger equation is an 
        eigenvalue equation. In mathematics, diagonalization of the matrix allows us
        to obtain its eigenvalues and eigenfunctions. You can read more 
        about the theory on 
        <a href="https://en.wikipedia.org/wiki/Diagonalizable_matrix">Wikipedia</a>.
    </details>
    </li>
</ol>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
%matplotlib widget

from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eigh_tridiagonal
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, jslink, VBox, HBox, Button, Label

In [ ]:
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
ixx = 0

def singlewell_potential(x, width, depth):
    x1 = zeros(len(x))
    for i in range(len(x)):
        if x[i] > - width/2.0 and x[i] < width/2.0:
            x1[i] = depth
    return x1
    

def diagonalization(hbar, L, N, pot=singlewell_potential, width = 0.1, depth = 0.0):
    """Calculate sorted eigenvalues and eigenfunctions by diagonalization of the Hamiltonian matrix. 

       Input:
         hbar: reduced Planck constant
         L: set discretized interval as [-L,L] 
         N: number of grid points, i.e., size of the H matrix 
         pot: python function returning the value of the potential energy
         x0: center of the quantum well
         width: the width of the quantum well
         depth: the depth of the quantum well
       Ouput:
         ew: sorted eigenvalues (array of length N)
         ef: sorted eigenfunctions, ef[:,i] (size N*N)
         x:  grid points (array of length N)
         dx: grid spacing (called Delta in the theory above)
         V:  values of the potential discretized on the grid x (array of length N)
    """
    x = linspace(-L, L, N+2)[1:N+1]                 # grid points
    dx = x[1] - x[0]                                # grid spacing
    V = pot(x, width, depth)
    z = hbar**2 /2.0/dx**2                         # coefficient

    ew, ef = eigh_tridiagonal(V+2.0*z, -z*ones(N-1))
    ew = ew.real                                    # real part of the eigenvalues
    ind = argsort(ew)                               # Indices f. sort. Array
    ew = ew[ind]                                    # Sort the ew by ind
    ef = ef[:, ind]                                 # Sort the columns 
    ef = ef/sqrt(dx)                                # Correct standardization 
    return ew, ef, x, dx, V


def plot_eigenfunctions(ax, ew, ef, x, V, width=1, Emax=0.05, fak= 5.0):
    """Create the full plot.
    
    Plot the lowest squared eigenfunctions 'ef' at the level of the eigenvalues
    'ew' in the plot area 'ax', and the potential 'V(x)'.
    """
    
    fak = fak/100.0
    
    ax[0].axhspan(0.0, Emax, facecolor='lightgrey')
    
    ax[0].set_xlim([min(x), max(x)])
    ax[0].set_ylim([min(V)-0.05, Emax])
    
    ax[0].set_xlabel(r'$x/a$', fontsize = 10)
    ax[0].set_ylabel(r'$V(x)$ and squared eigenfunctions', fontsize = 10)
    
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V)-0.05, Emax])
    
    ax[1].yaxis.set_label_position("right")
    ax[1].yaxis.tick_right()
    
    ax[1].get_xaxis().set_visible(False)
    ax[1].set_ylabel(r'$\rm{\ Eigenvalues}$', fontsize = 10)
    
    indmax = sum(ew<=0.0)                       
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[0].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])
        ax[1].plot(x, x*0.0+ew[i], linewidth=width[i]+2.5, color=colors[i%len(colors)])
        
    ax[0].plot(x, V, c='k', linewidth=1.6)

In [ ]:
mu = 0.06                                            # Potential parameter
L = 1.5                                              # x range [-L,L]
N = 200                                              # Number of grid points
hbar = 0.06                                          # Reduced Planck constant
sigma_x = 0.1                                        # Width of the Gaussian function
zeiten = linspace(0.0, 10.0, 400)                    # time


swidth = FloatSlider(value = 1.2, min = 0.1, max = 2.0, description = 'Width: ')
sdepth = FloatSlider(value = -0.2, min = -1.0, max = 0.0, step = 0.05, description = 'Depth: ')
sfak = FloatSlider(value = 5, min = 1.0, max = 10.0, step = 1.0, description = r'Zoom factor: ')
update = Button(description="Show all")

width = 1.2
depth = -0.2
fak = 5.0

ew, ef, x, dx, V = diagonalization(hbar, L, N, width = width, depth = depth)
    
fig, ax = plt.subplots(1, 2, figsize=(7,5), gridspec_kw={'width_ratios': [10, 1]})
fig.canvas.header_visible = False
fig.canvas.layout.width = "750px"

fig.suptitle(r'Numerical Solution ($\psi^2$) of the Schrödinger Equation for a 1D Quantum Well', fontsize = 12)
plot_eigenfunctions(ax, ew, ef, x, V)

def on_update_click(b):
    for i in ax[0].lines:
        i.set_alpha(1.0)
    for i in ax[1].lines:
        i.set_alpha(1.0)
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

def on_width_change(change):
    global ew, ef, x, dx, V
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    ew, ef, x, dx, V = diagonalization(hbar, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunctions(ax, ew, ef, x, V, fak = sfak.value)

def on_depth_change(change):
    global ew, ef, x, dx, V
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    ew, ef, x, dx, V = diagonalization(hbar, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunctions(ax, ew, ef, x, V, fak = sfak.value)
    
def on_xfak_change(change):
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    plot_eigenfunctions(ax, ew, ef, x, V, fak = sfak.value)

def on_press(event):
    global ann, ann1, ixx
    
    ixx = min(enumerate(ew), key = lambda x: abs(x[1]-event.ydata))[0]
    
    for i in range(len(ax[1].lines)):
        ax[0].lines[i].set_alpha(0.1)
        ax[1].lines[i].set_alpha(0.1)
        ax[0].lines[i].set_linewidth(1.1)
        
    ax[0].lines[ixx].set_alpha(1.0)
    ax[1].lines[ixx].set_alpha(1.0)
    ax[0].lines[ixx].set_linewidth(2.0)
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass
    
    ann = ax[0].annotate(s = 'n = ' + str(ixx+1), xy = (0, ew[ixx]), xytext = (-0.15, ew[ixx]), xycoords = 'data', color='k', size=15)
    ann1 = ax[1].annotate(s = str("{:.3f}".format(ew[ixx])), xy = (0, ew[ixx]), xytext = (-1.2, ew[ixx]+0.005), xycoords = 'data', color='k', size=9)

cid = fig.canvas.mpl_connect('button_press_event', on_press)

swidth.observe(on_width_change, names = 'value')
sdepth.observe(on_depth_change, names = 'value')
sfak.observe(on_xfak_change, names = 'value')
update.on_click(on_update_click)

label1 = Label(value="(click on a state to select it)");

display(HBox([swidth, sdepth]), sfak, HBox([update, label1]))

* **Width:** the width of the quantum well.
* **Depth:** the depth of the quantum well.
* **Zoom factor:** the zoom factor for the magnitude of the eigenfunctions.

<details>
    <summary style="font-size: 22px;"><b>Legend</b></summary>

<p style="text-align: justify;font-size:15px"> 
    There are three sliders you can tune interactively. Two of them can adjust the width and depth of the
    potential well. The zoom factor slider is used to adjust the normalization factor of the wavefunctions. 
    As the normalization of the wavefunction is arbitrary,
    this factor is used for visualization purposes only, to avoid overlap of the wavefunctions while at the
    same time avoiding that the wavefunctions are so narrow that you cannot appreciate their shape.
</p>

<p style="text-align: justify;font-size:15px"> 
    In the main visualization part, there are two subplots.
    The wide figure on the left shows the well potential 
    and the square of the eigenfunctions $\psi^2$ of different states, plotted
    at the energy of the corresponding eigenvalues. 
    The narrow figure on the right shows the eigenvalues.
    You can select either an eigenfunction or an eigenvalue by clicking on it in one of the 
    two subplots. 
    The eigenfunction and corresponding eigenvalue will be highlighted, and the corresponding
    eigenenergy and value of the index $n$ will be shown. All other eigenstates will
    fade out.
    By clicking the "Show all" button, you can reset the plot and show all states again.
</p>

</details>